In [0]:
%run ../includes/configuration

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
race_reuslts_df = spark.read.parquet(f'{presentation_folder_path}race_results')\
.filter(f"file_date='{v_file_date}'")

In [0]:
race_year_list = df_column_to_list(race_reuslts_df, 'race_year')

In [0]:
from pyspark.sql.functions import sum, when, count, col
race_reuslts_df = spark.read.parquet(f'{presentation_folder_path}race_results').\
filter(col("race_year").isin(race_year_list))

In [0]:
from pyspark.sql.functions import sum, when, count, col,desc
constructor_standing_df = race_reuslts_df\
.groupby('race_year', 'team')\
.agg(sum('points').alias('total_points'),
count(when (col('position')==1, True)).alias('wins'))

In [0]:
from pyspark.sql.functions import rank

In [0]:
from pyspark.sql.window import Window

constructor_rank_spec = Window.partitionBy('race_year').orderBy(desc('total_points'), desc('wins'))
final_df = constructor_standing_df.withColumn("rank", rank().over(constructor_rank_spec))

In [0]:
overwrite_partition(final_df, 'race_year', 'f1_presentation', 'constructor_standings')

In [0]:
%sql
SELECT race_year, count(1)
FROM f1_presentation.constructor_standings
GROUP BY race_year
ORDER BY race_year DESC

race_year,count(1)
2021,10
2020,10
2019,10
2018,10
2017,10
2016,11
2015,10
2014,11
2013,11
2012,12
